# Импортирование библиотек/данных

In [ ]:
import pandas as pd
import json

In [793]:
with open('data_file_it.json') as f:
    data = json.load(f)

In [794]:
#data[0]['education'][1]['name']
#data[0]
#data[17]['skills']

In [795]:
gender = []
area = []
title = []
salary =[]
edu = []
exp =[]
skill_set = []
skills = []

for resume in data:
  gender.append(resume['gender'])
  area.append(resume['area'])
  title.append(resume['title'])
  salary.append(resume['salary']['amount'])
  edu.append(resume['education_level'])
  skill_set.append(resume['skill_set'])
  skills.append(resume['skills'])

  exp_temp = []
  for i in range(len(resume['experience'])):
    exp_temp.append(resume['experience'][i]['position'])
  exp.append(exp_temp)
 

In [796]:
df = pd.DataFrame([gender, area, title, salary, edu, skill_set, skills, exp]).transpose()

In [797]:
df.head()

,0,1,2,3,4,5,6,7
0,Мужчина,Москва,Специалист по машинному обучению,120000,Высшее образование (Магистр),"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++]
1,Мужчина,Москва,Разработчик React-Native JavaScript (Redux),None,Высшее образование,"[HTML, CSS, React-Native, JS6, Git, FireBase, ...","Из резюме удален весь нерелевантный опыт, чтоб...","[Разработчик React Native, Разработчик React N..."
2,Мужчина,Санкт-Петербург,ИТ-специалист,None,Высшее образование,"[HTML, CSS, PHP, Сборка ПК, Настройка ПК, MySQ...",Окончил университет СПБГУТ в Санкт-Петербурге ...,"[Специалист технической поддержки, ИТ Специали..."
3,Мужчина,Санкт-Петербург,Программист C#,None,Среднее образование,"[C#, Unity, Godot, SQL, MySQL, Git, .NET Frame...",Примеры моего кода и прочего\nhttps://github.c...,"[Оператор call-центра, Диспетчер технической п..."
4,Женщина,Санкт-Петербург,Frontend-разработчик (JavaScript) junior,30000,Высшее образование,"[HTML5, CSS3, JavaScript, jQuery, CSS, Git, HT...",- Дипломы колледжа и бакалавра с отличием (кра...,"[Junior разработчик веб-интерфейсов, Оператор..."


In [798]:
#gender, area, title, salary, edu, skill_set, skills, exp, spec_name, prior
df.rename(columns={0:'gender', 1:'area', 2:'title', 3:'salary',
                   4: 'education', 5:'skill_set', 6: 'skills', 7:'exp', 8:'spec_name'
                    }, inplace = True)

# Очистка данных


In [799]:
df['education'].value_counts()

Высшее образование                    198
Неоконченное высшее образование        51
Высшее образование (Магистр)           36
Высшее образование (Бакалавр)          34
Среднее специальное образование        34
Образование                            16
Higher education                        5
Среднее образование                     4
Higher education (bachelor)             4
Высшее образование (Кандидат наук)      3
Education                               2
Secondary special education             2
Secondary education                     1
Name: education, dtype: int64

In [800]:
#Приведение поля с образованием к единому виду
df.loc[df.education =='Higher education', 'education'] = 'Высшее образование' # берем поле education, где education = Higher education
df.loc[df.education =='Высшее образование (Магистр)', 'education'] = 'Высшее образование'
df.loc[df.education =='Высшее образование (Бакалавр)', 'education'] = 'Высшее образование'
df.loc[df.education =='Higher education (bachelor)', 'education'] = 'Высшее образование'
df.loc[df.education =='Высшее образование (Кандидат наук)', 'education'] = 'Высшее образование'
df.loc[df.education =='Secondary special education', 'education'] = 'Среднее образование'
df.loc[df.education =='Secondary education', 'education'] = 'Среднее образование'
df.loc[df.education =='Education', 'education'] = 'Среднее образование'

In [801]:
df['education'].value_counts()

Высшее образование                 280
Неоконченное высшее образование     51
Среднее специальное образование     34
Образование                         16
Среднее образование                  9
Name: education, dtype: int64

In [802]:
df['area'].value_counts()

Москва             148
Санкт-Петербург     64
Краснодар           14
Челябинск            8
Екатеринбург         8
                  ... 
Долгопрудный         1
Новороссийск         1
Раменское            1
Ufa                  1
Вологда              1
Name: area, Length: 90, dtype: int64

In [803]:
#Приведение поля с городом к единому виду
df['area'] = df['area'].apply(lambda x: x if x=='Москва' or x=='Санкт-Петербург' else 'Регионы')

In [804]:
df['area'].value_counts()

Регионы            178
Москва             148
Санкт-Петербург     64
Name: area, dtype: int64

In [805]:
df['gender'].value_counts()

Мужчина    347
Женщина     29
Male        12
Female       2
Name: gender, dtype: int64

In [806]:
df.loc[df.gender =='Male', 'gender'] = 'Мужчина'
df.loc[df.gender =='Female', 'gender'] = 'Женщина'

In [807]:
df['gender'].value_counts()

Мужчина    359
Женщина     31
Name: gender, dtype: int64

In [808]:
df = df.join(pd.get_dummies(df['gender']))

In [809]:
df.drop('gender', axis = 1, inplace = True)

In [810]:
areas = list(df['area'].unique())
#df['area'].unique()
#df['area'].value_counts()
#df['area'].value_counts().index
for i in range(len(areas)):
  df.loc[df['area']==areas[i], 'area'] = i

In [811]:
titles = list(df['title'].unique())
for i in range(len(titles)):
  df.loc[df['title']==titles[i], 'title'] = i

In [812]:
edus = list(df['education'].unique())
for i in range(len(edus)):
  df.loc[df['education']==edus[i], 'education'] = i

In [813]:
df.head()

,area,title,salary,education,skill_set,skills,exp,Женщина,Мужчина
0,0,0,120000,0,"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++],0,1
1,0,1,None,0,"[HTML, CSS, React-Native, JS6, Git, FireBase, ...","Из резюме удален весь нерелевантный опыт, чтоб...","[Разработчик React Native, Разработчик React N...",0,1
2,1,2,None,0,"[HTML, CSS, PHP, Сборка ПК, Настройка ПК, MySQ...",Окончил университет СПБГУТ в Санкт-Петербурге ...,"[Специалист технической поддержки, ИТ Специали...",0,1
3,1,3,None,1,"[C#, Unity, Godot, SQL, MySQL, Git, .NET Frame...",Примеры моего кода и прочего\nhttps://github.c...,"[Оператор call-центра, Диспетчер технической п...",0,1
4,1,4,30000,0,"[HTML5, CSS3, JavaScript, jQuery, CSS, Git, HT...",- Дипломы колледжа и бакалавра с отличием (кра...,"[Junior разработчик веб-интерфейсов, Оператор...",1,0


In [814]:
import numpy as np
#поиск по скиллам
def skill_search(col, skills):
  """
  Функция для поиска ключевых слов в резюме
  """
  for skill in skills:
    df[skill] = 0 # создание столбца с названием skill 
    skilled_ids = df[col].apply(lambda x: x if skill in str(x).lower() 
                                else np.nan).dropna().index # все айди людей с этим скиллом
    df.loc[df.index.isin(skilled_ids), skill] = 1 # если индекс равен ид из skilled_ids, в столбце skill получают 1


In [815]:
skills_set_for_success = ['машинное обучение', 'data mining', 'machine learning', 'sql', 'html', 'css']
col = 'skill_set'
skill_search(col, skills_set_for_success)

In [816]:
#Создание столбца success, которое означает, прошел кандидат или нет. Набираем DSов, аналитиков/разработчиков БД, а также верстальщиков сайтов)
conditions = [(df['машинное обучение'] == 1),
              (df['data mining'] == 1),
              (df['machine learning'] == 1),
              (df['sql'] == 1),
              (df['html'] == 1),
              (df['css'] == 1)
             ]
             
values = [1, 1, 1, 1, 1, 1] 

df['success'] = 0
df['success'] = np.select(conditions, values)

df.drop(['машинное обучение', 'data mining', 'machine learning', 'sql', 'html', 'css'], axis = 1, inplace = True)

In [817]:
skills_for_candidates = ['английский', 'github']
col = 'skills'
skill_search(col, skills_for_candidates)

In [818]:
exp = ['программист c', '++', 'react']
col = 'exp'
skill_search(col, exp)

In [819]:
skills = ['машинное обучение', 'sql', 'data mining', 'python', 'pandas', 'numpy']
col = 'skill_set'
skill_search(col, skills)

In [820]:
df.drop(['skill_set', 'skills', 'exp'], axis = 1, inplace = True)

In [821]:
df.fillna(0,inplace = True)

#Создание модели и её обучение

In [822]:
x = df.drop('success', axis = 1)
y = df['success']

In [823]:
from sklearn.model_selection import train_test_split

x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size = .3, random_state =1)

In [824]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(x_tr, y_tr)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [825]:
proba = pd.DataFrame(rfc.predict_proba(x_ts))[1]
#predict выводил либо 0, либо 1, predict_proba выводит вероятность для каждого пройти или не пройти

In [826]:
rfc.predict(x_ts)

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1])

In [827]:
proba[:20]

0     1.00
1     1.00
2     0.22
3     0.43
4     1.00
5     0.53
6     0.44
7     0.95
8     0.98
9     0.71
10    0.76
11    0.97
12    0.94
13    1.00
14    0.98
15    0.98
16    1.00
17    0.70
18    0.46
19    0.94
Name: 1, dtype: float64

In [828]:
from sklearn.metrics import accuracy_score # количество попаданий

accuracy_score(rfc.predict(x_ts), y_ts) # в ~80% случаев модель правильно подбирает кандидатов в команду

0.8034188034188035